In [1]:
import numpy as np 

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import decomposition
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# different regressor test

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import NuSVR, SVR, LinearSVR 
from sklearn.linear_model import Lasso, Ridge, LinearRegression

import pandas as pd # process txt
from scipy.io import loadmat
from PIL import Image

from collections import defaultdict

In [2]:
folder = "G:/dataset/sun/"

standard_path = "G:/dataset/standard_split/SUN/"
proposed_path = "G:/dataset/proposed_split/SUN/"


cls_to_idx = {}
cnt = 0 
with open(standard_path + "allclasses.txt", "r", encoding='utf-8') as f: # TBD
     for row in f.readlines():
         row = row.rstrip()
         cls_to_idx[row] = cnt
         cnt = cnt + 1

sstrain, sstest = [], []
pstrain, pstest = [], []

# Standard Split 
with open(standard_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstrain.append(cls_to_idx[row])

with open(standard_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstest.append(cls_to_idx[row])

print("standard_split:", len(sstrain), len(sstest))
# transform List(str) -> List(int)

# Proposed Split
with open(proposed_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstrain.append(cls_to_idx[row])

with open(proposed_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstest.append(cls_to_idx[row])

print("proposed_split:", len(pstrain), len(pstest))

# Random Train & Test Class Split

train_class, test_class = [], [] 
X_class = list(range(717)) # label start from 0, you can adjust it
train_class, test_class = train_test_split(X_class, test_size=0.2)

# len(train_class), len(test_class) == (573, 144)

train_class, test_class = sstrain, sstest
train_class, test_class = pstrain, pstest



standard_split: 645 72
proposed_split: 645 72


In [3]:
print(*test_class, sep = ", ")

3, 622, 420, 482, 695, 298, 152, 679, 103, 509, 674, 95, 711, 645, 681, 517, 145, 85, 138, 631, 216, 425, 53, 558, 423, 72, 99, 130, 10, 440, 24, 336, 353, 658, 57, 245, 381, 286, 75, 124, 650, 196, 262, 342, 74, 254, 246, 448, 529, 656, 259, 221, 635, 112, 38, 580, 715, 237, 32, 23, 493, 328, 471, 712, 184, 358, 508, 379, 710, 315, 158, 560


In [4]:
class sunRead:
    def __init__(self, p, train_split):
        """
        p: 数据集存放路径
        train_split: 给出训练集
        """
        X_class = list(range(717))
        train_class = train_split
        test_class = list(filter(lambda i: i not in train_class, X_class))
        self.path = p
        # labels
        yp = self.path + "custom-sun-labels.txt"
        y = np.loadtxt(yp, delimiter=" ", encoding='utf-8')
        # visual features 2048 dimensions
        xp = self.path + "sun-features.txt"
        x = np.loadtxt(xp, delimiter=" ", encoding='utf-8')
        i1 = np.isin(y, train_class)
        i2 = np.isin(y, test_class)
        self.X_train, self.X_test = x[i1], x[i2]
        self.y_train, self.y_test = y[i1], y[i2]
    def train_data(self):
        return self.X_train, self.y_train
    def test_data(self):
        return self.X_test, self.y_test
    def test(self): # 11788 all, add them to test
        print(self.X_train.shape, self.y_train.shape)
        print(self.X_test.shape, self.y_test.shape)

sunReader = sunRead(folder, train_class)
sunReader.test()
# custom
# standard 12900 + 1440

(12900, 2048) (12900,)
(1440, 2048) (1440,)


In [5]:
X_train, y_train = sunReader.train_data()
X_test, y_test = sunReader.test_data()
#X_train.shape, y_train.shape, X_test.shape, y_test.shape
pca_d = 500

exemPCA = decomposition.PCA(n_components=pca_d)
exemPCA.fit(X_train)
X_train = exemPCA.transform(X_train)
X_test = exemPCA.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((12900, 500), (12900,), (1440, 500), (1440,))

In [6]:
# group up PCA projections

exem_train_group = defaultdict(list)

for c in train_class:
    exem_train_group[c] = []
        
for x, y in zip(X_train, y_train):
    exem_train_group[y].append(x)


# Average
exem_train = {}
std_train = {}
k = 0

for item in exem_train_group.items():
    y, ary = item
    exem_train[y] = np.mean(ary, axis=0) # Key Sentence
    std_train[y] = np.std(ary, axis=0)

del exem_train_group

# semantic embedding of SUN, 717 scenes, 102 attributes
# fork s.e. from essay "evaluation"
#se = folder + "sun_se_continuous.txt"

att = loadmat(standard_path + "att_splits.mat")
#semat = att['att'].T # transpose
semat = att['att_original']


# class to index, the same
a_c_train, a_c_test = semat[train_class], semat[test_class]
v_c_train = [exem_train[i] for i in train_class] # dict uses classes to index
sd_train = [std_train[i] for i in train_class]
sd_train = np.mean(sd_train, axis=0)

In [7]:
semat

array([[0.        , 0.08333333, 0.08333333, ..., 0.01666667, 0.05      ,
        0.01666667],
       [0.01666667, 0.78333333, 0.71666667, ..., 0.01666667, 0.06666667,
        0.01666667],
       [0.        , 0.06666667, 0.18333333, ..., 0.01666667, 0.08333333,
        0.08333333],
       ...,
       [0.        , 0.08333333, 0.08333333, ..., 0.01666667, 0.33333333,
        0.01666667],
       [0.        , 0.2       , 0.1       , ..., 0.03333333, 0.08333333,
        0.06666667],
       [0.05      , 0.        , 0.01666667, ..., 0.05      , 0.11666667,
        0.        ]])

In [8]:
np.array(a_c_train).shape, np.array(v_c_train).shape, np.array(sd_train).shape

((645, 102), (645, 500), (500,))

In [9]:
# c = 2 ^ -3, nu = 2^0, gamma = 2 ^ -1
regress_group = []
k = 0
for j in range(pca_d):
    X = a_c_train # 
    y = [vc[j] for vc in v_c_train]
    regressor = Ridge()
    regressor.fit(X, y)
    regress_group.append(regressor)

v_c_test = np.zeros((72, pca_d)) # 提前定义好exemplar矩阵, 
# 对每一个维度进行预测
for j in range(pca_d):
    v_c_test[:, j] =  regress_group[j].predict(a_c_test) # 10 dimension , assign to column

v_c_test.shape, len(v_c_train), len(v_c_train[0])


exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))


1NN:0.6048611111111111 
1NNs:0.6284722222222222


In [10]:
regress_group[4].predict(a_c_test), regress_group[4].predict(a_c_test).shape, regress_group[5].predict(a_c_test)

(array([-1.91394373,  6.15955633,  4.16936191,  2.71024412,  0.90121109,
         1.53513074, -4.27987195, -0.48029197, -1.23852818,  3.95271928,
         2.24909257,  2.14895236,  1.24080688, -1.1458303 , -9.84945923,
        -5.53675212,  0.42637562,  0.29126825,  3.16931042, 10.5324946 ,
        -4.82219387, -1.58007511,  5.09256182, -8.89646887,  8.40320758,
        -9.36061805,  0.98293545, -1.57453961,  1.72512778,  7.27115049,
         5.50767091,  3.90561563, -0.68071308, -0.49173012,  2.55778431,
        -2.58620555, -4.97099022, -1.53825112, -2.76908637, -1.89445717,
         0.6551227 , -3.88476935,  1.44584843,  2.23864585,  0.97534534,
         0.70201031,  0.62167916, -1.58063794, -0.11265385,  2.12906199,
        -3.92280259,  0.31006548, -2.19845982, -5.78770454,  8.61280647,
        -0.45100098,  5.61953623,  2.70893663,  0.09773444,  1.04984553,
        -6.86301107,  4.1153905 , -5.08518191, -2.40147621, -4.01201145,
         2.07371221, -1.31131453, -4.14664062, -2.2

In [11]:
## Save Exemplars and visual features of test class
save_path = "./ridge_exem/"
np.save(save_path + "exem_test.npy", np.array(exem_X))
np.save(save_path + "X_test.npy", X_test)
np.savetxt(save_path + "y_test.txt", np.array(y_test, dtype=int), fmt='%s', encoding='utf-8')

In [12]:
# accuracy = neigh.score(X_test, y_test) # deprecated
# accuracy
# 4.8
# ZSL 0.44513888888888886
# GZSL 0.15381944444444445

# 4.13 Standard Split
# ZSL 0.575
# GZSL 0.14166666666666666

# 4.14 Standard Split with better ResNet101 features
# ZSL 
# 0.6215277777777778 nu-SVR C=2
# 0.6013888888888889 SVR
# 0.5972222222222222 nu-SVR default
# 0.5736111111111111 LinearSVR
# 0.6305555555555555 Lasso
# 0.5854166666666667 Ridge

# 4.16 Proposed Split 
# 0.5854166666666667 Ridge
# 0.597 nu-svr default
# 0.6215277777777778 nu-svr C=2